---
Title: Introduction To Bitmap File Formats.
Date: 2024-03-01 12:00
Lang: zh-cn
Category: FileFormats
---

# 介绍

Windows 的图形设备接口(GDI)，在 Windows 1.0 开始便支持位图。但此时的位图并不是此文介绍的BMP图像，而是"GDI位图对象"，也被称为"设备相关位图"`(Device-dependent bitmap, DDB)`。

到了20世纪80年代中期，微软与IBM合作开发的操作系统 `OS/2 1.1` 版本中定义了 `DIB` 格式，这就是早期 `BMP` 图像的起源。`OS/2 1.1` 于 1988 年发布，是第一个拥有像 Windows 一样的图形用户界面`(Presentation Manager, PM)`的 `OS/2` 版本。PM 包含一套图形编程接口，其中定义了 `BMP` V1版本的位图格式。

> 参考文档:
- [微软开发文档--位图存储](https://learn.microsoft.com/en-us/windows/win32/gdi/bitmap-storage)


## BMP文件存储结构

BMP图像在磁盘存储的比特分为几个部分: `BITMAPFILEHEADER`, `BITMAPINFOHEADER`, `BITMAPV4HEADER`/`BITMAPV5HEADER` 结构组成位图信息头部结构，以及跟随之后的可选 `RGBQUAD` 结构数组(也称颜色表)和实际的图像数据/索引数组。

